# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow import keras
from datetime import datetime
from keras.optimizers import Adam
from keras.regularizers import L2
from keras.utils import plot_model
from keras.models import Sequential
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from keras.metrics import Precision, Recall
from sklearn.metrics import confusion_matrix
from tensorflow_addons.metrics import F1Score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from keras.layers import LSTM, Dense, Dropout,MaxPooling1D,Flatten,Conv1D, GRU, Bidirectional

# Processing functions

In [ ]:
def ConfusionMatrix(model_name,model,X_test,y_test,show=False,OurDataset=True):
    y_test_original = np.argmax(y_test, axis=1)
    y_pred = model.predict(X_test)
    predicted_label_indices = np.argmax(y_pred, axis=1) # get the index of the highest probability for each sample
    # predicted_label_indices
    cm = confusion_matrix(y_test_original, predicted_label_indices)
    target_names=[]
    if OurDataset:
        target_names=['Bad-Double-Arm-Ball','Bad-Flamingo-Movement','Bad-Flamingo-Stand','Bad-Side-Leg-Raise-Movement','Bad-Single-Leg-Standing','Bad-Squatting','Bad-Tummy-Twist',
                    'Good-Double-Arm-Ball','Good-Flamingo-Movement','Good-Flamingo-Stand','Good-Side-Leg-Raise-Movement','Good-Single-Leg-Standing','Good-Squatting','Good-Tummy-Twist']
    else:
        target_names=['deep squat', 'hurdle step', 'inline lunge', 'side lunge', 'sit to stand',
                    'standing active straight leg raise','standing shoulder abduction', 'standing shoulder extension', 'standing shoulder internal-external rotation', 'standing shoulder scaption',
                    'incorrect deep squat', 'incorrect hurdle step', 'incorrect inline lunge', 'incorrect side lunge', 'incorrect sit to stand',
                    'incorrect standing active straight leg raise','incorrect standing shoulder abduction', 'incorrect standing shoulder extension', 'incorrect standing shoulder internal-external rotation', 'incorrect standing shoulder scaption']
    labels = target_names
    sns.set(font_scale=2.5)  # Adjust the font size if necessary
    plt.figure(figsize=(32, 32))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - ' + model_name)
    plt.tight_layout()
    plt.savefig('./Confusion Matrices/'+model_name+'.png')
    if show == True:
        plt.show()
    plt.close()

def ReadOneData(DropColumn,input_directory):
    data = pd.read_csv(input_directory)
    x=data.drop(DropColumn, axis=1).values
    y=data[DropColumn].values
    return x,y

def ReadThreeData(DropColumn,input_directory):
    x = []
    y = []
    for file_name in os.listdir(input_directory):
        if file_name.endswith('.csv'):
            input_file_path = os.path.join(input_directory, file_name)
            data = pd.read_csv(input_file_path)
            last_x = normalize(data.drop(DropColumn, axis=1).values).astype('float16')
            last_y = data[DropColumn].values[0]
            x.append(last_x)
            y.append(last_y)
    return x, y

def normalize(X):
    sc = MinMaxScaler(feature_range=(0, 1))
    X_normalized = sc.fit_transform(X)
    return X_normalized

def reshapeData(X, time_steps, no_features):
    X = np.reshape(X, (X.shape[0], no_features, time_steps))
    return X

def encodeLabels(y):
    label_encoder = LabelEncoder()
    Y_train_labeled = label_encoder.fit_transform(y)
    unique = np.unique(Y_train_labeled)
    onehot_encoder = OneHotEncoder(sparse_output=False, categories='auto')
    Y_train_encoded = onehot_encoder.fit_transform(y.reshape(-1, 1))
    return unique, Y_train_encoded

def saveModel(model_name, model):
    models_folder = './Models/'
    date = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    savepath = f'{models_folder}{model_name}-{date}.h5'
    model.save(savepath)
    return savepath

# Models architecture

In [ ]:
def GRU_Model(no_features,timesteps,no_classes):
    model = Sequential()
    model.add(GRU(units=256, input_shape=(timesteps,no_features), return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(units=1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(units=no_classes, activation='softmax'))

    optimizer = Adam(learning_rate= 0.0012)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), F1Score(num_classes=no_classes,name='f1')], optimizer=optimizer)
    return model

def BILSTM_Model(no_features,timesteps,no_classes):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=128, return_sequences=True), input_shape=(timesteps,no_features)))
    model.add(Dropout(0.2))

    model.add(Flatten())

    model.add(Dense(units=1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(units=no_classes, activation='softmax'))

    optimizer = Adam(learning_rate=0.0012)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), F1Score(num_classes=no_classes,name='f1')], optimizer=optimizer)
    return model

def LSTM_Model(no_features,timesteps,no_classes):
    lstm_units= 256
    dropout_rate= 0.2
    dense_units= 512
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True,input_shape=(timesteps,no_features)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(units=dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=no_classes, activation='softmax'))

    optimizer = Adam(learning_rate= 0.0012)
    model.compile(loss='categorical_crossentropy',metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), F1Score(num_classes=no_classes,name='f1')],optimizer=optimizer)
    return model

def CNN_LSTM_Model(no_features,timesteps, no_classes):
    model = Sequential()
    conv_filters= 128
    kernel_size= 8
    lstm_units= 256
    dropout_rate= 0.2
    regularizer_strength= 0.00279
    dense_layers= 2
    dense_units= 525
    num_conv_layers= 2
    learning_rate= 0.0012
    for _ in range(num_conv_layers):
        conv_layer = Conv1D(filters=conv_filters, kernel_size=kernel_size, activation='relu', input_shape=(timesteps,no_features))
        conv_layer.kernel_regularizer = keras.regularizers.l2(regularizer_strength) 
        conv_layer.bias_regularizer = L2(regularizer_strength)
        model.add(conv_layer)
        model.add(MaxPooling1D(2))
        model.add(Dropout(dropout_rate))
    
    model.add(LSTM(units=lstm_units, return_sequences=True,input_shape=(timesteps,no_features)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    for _ in range(dense_layers):
        model.add(Dense(units=dense_units, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(units=no_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), F1Score(num_classes=no_classes,name='f1')])
    return model

def CNN_GRU_Model(no_features,timesteps, no_classes):
    model = Sequential()
    conv_filters= 128
    kernel_size= 8
    dropout_rate= 0.2
    regularizer_strength= 0.0001
    num_conv_layers= 2
    learning_rate= 0.0001
    for _ in range(num_conv_layers):
        conv_layer = Conv1D(filters=conv_filters, kernel_size=kernel_size, activation='relu', input_shape=(timesteps,no_features))
        conv_layer.kernel_regularizer = keras.regularizers.l2(regularizer_strength) 
        conv_layer.bias_regularizer=L2(regularizer_strength)
        model.add(conv_layer)
        model.add(MaxPooling1D(2))
        model.add(Dropout(dropout_rate))
    
    model.add(GRU(units=512, input_shape=(timesteps,no_features), return_sequences=True))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(units=512, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(units=256, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(units=128, activation="relu"))
    model.add(Dense(units=no_classes, activation="softmax"))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate),loss='categorical_crossentropy',metrics=['accuracy', Precision(name='precision'), Recall(name='recall'), F1Score(num_classes=no_classes,name='f1')])
    return model

def ModelSummary(no_features, timesteps, no_classes, Dim):
    model = GRU_Model(no_features, timesteps, no_classes)
    plot_model(model, to_file=f'../Documents/GRU_Model{Dim}.png', show_shapes=True, show_layer_names=True)
    
    model = BILSTM_Model(no_features, timesteps, no_classes)
    plot_model(model, to_file=f'../Documents/BILSTM_Model{Dim}.png', show_shapes=True, show_layer_names=True)
    
    model = LSTM_Model(no_features, timesteps, no_classes)
    plot_model(model, to_file=f'../Documents/LSTM_Model{Dim}.png', show_shapes=True, show_layer_names=True)
    
    model = CNN_LSTM_Model(no_features, timesteps, no_classes)
    plot_model(model, to_file=f'../Documents/CNN_LSTM_Model{Dim}.png', show_shapes=True, show_layer_names=True)
    
    model = CNN_GRU_Model(no_features, timesteps, no_classes)
    plot_model(model, to_file=f'../Documents/CNN_GRU_Model{Dim}.png', show_shapes=True, show_layer_names=True)

# Run functions

In [ ]:
def Train(X,y,no_features,time_steps,no_classes,epochs,batch_size,Dim,IsOur=True):
    #Train Test Splitting
    All_X_train, X_test, All_y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #Train Validation Splitting
    X_train, X_valid, y_train, y_valid = train_test_split(All_X_train, All_y_train, test_size=0.2, random_state=42)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    if Dim!=3:
        #Reshaping Data
        X_train = reshapeData(X_train,time_steps,no_features)
        X_valid = reshapeData(X_valid,time_steps,no_features)
        X_test = reshapeData(X_test,time_steps,no_features)
    # LSTM model
    model = LSTM_Model(time_steps,no_features, no_classes)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])
    accuracy = model.evaluate(X_valid, y_valid)[1]* 100
    model_name = f'LSTM-{Dim}D-{accuracy:.2f}%' if IsOur else f'UIPRMD-LSTM-{Dim}D-{accuracy:.2f}%'
    saveModel(model_name,model)
    ConfusionMatrix(model_name,model,X_test,y_test) if IsOur else ConfusionMatrix(model_name,model,X_test,y_test,OurDataset=False)
    model.reset_states()
    
    #CNN-LSTM model
    model = CNN_LSTM_Model(time_steps,no_features, no_classes)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])
    accuracy = model.evaluate(X_valid, y_valid)[1]* 100
    model_name = f'CNN-LSTM-{Dim}D-{accuracy:.2f}%' if IsOur else f'UIPRMD-CNN-LSTM-{Dim}D-{accuracy:.2f}%'
    saveModel(model_name,model)
    ConfusionMatrix(model_name,model,X_test,y_test) if IsOur else ConfusionMatrix(model_name,model,X_test,y_test,OurDataset=False)
    model.reset_states()
    
    #GRU model
    model = GRU_Model(time_steps,no_features, no_classes)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])
    accuracy = model.evaluate(X_valid, y_valid)[1]* 100
    model_name = f'GRU-{Dim}D-{accuracy:.2f}%' if IsOur else f'UIPRMD-GRU-{Dim}D-{accuracy:.2f}%'
    saveModel(model_name,model)
    ConfusionMatrix(model_name,model,X_test,y_test) if IsOur else ConfusionMatrix(model_name,model,X_test,y_test,OurDataset=False)
    model.reset_states()
    
    #CNN-GRU model
    model = CNN_GRU_Model(time_steps,no_features, no_classes)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])
    accuracy = model.evaluate(X_valid, y_valid)[1]* 100
    model_name = f'CNN-GRU-{Dim}D-{accuracy:.2f}%' if IsOur else f'UIPRMD-CNN-GRU-{Dim}D-{accuracy:.2f}%'
    saveModel(model_name,model)
    ConfusionMatrix(model_name,model,X_test,y_test) if IsOur else ConfusionMatrix(model_name,model,X_test,y_test,OurDataset=False)
    model.reset_states()
    
    #BILSTM model
    model = BILSTM_Model(time_steps,no_features, no_classes)
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])
    accuracy = model.evaluate(X_valid, y_valid)[1]* 100
    model_name = f'BILSTM-{Dim}D-{accuracy:.2f}%' if IsOur else f'UIPRMD-BILSTM-{Dim}D-{accuracy:.2f}%'
    saveModel(model_name,model)
    ConfusionMatrix(model_name,model,X_test,y_test) if IsOur else ConfusionMatrix(model_name,model,X_test,y_test,OurDataset=False)
    model.reset_states()

# 1D Run on Collected DS

In [ ]:
# def Run():
#     DropColumn='Label'
#     input_directory = './CSVS/CSV_FeatureEngineering.csv'
#     x, y=ReadOneData(DropColumn,input_directory)
#     x=normalize(x)
#     unique,y=encodeLabels(y)
#     no_features = x.shape[1]
#     no_classes = len(unique)
#     timesteps = 1
#     epochs=50
#     batch_size=41
#     ModelSummary(timesteps, no_features, no_classes, 1)
#     Train(x, y, no_features, timesteps, no_classes, epochs, batch_size, 1)
# Run()

# 1D Run on UI_PRMD DS

In [ ]:
# def Run():
#     DropColumn='Label'
#     input_directory='./CSVS/CSV_STD.csv'
#     x, y=ReadOneData(DropColumn,input_directory)
#     unique,y=encodeLabels(y)
#     no_features = x.shape[1]
#     no_classes = len(unique)
#     time_steps = 1
#     epochs=50
#     batch_size=100
#     ModelSummary(timesteps, no_features, no_classes, 1)
#     Train(x, y, no_features, time_steps, no_classes, epochs, batch_size, 1,False)
# Run()

# 3D Run on Collected DS

In [ ]:
# def Run():
#     DropColumn='Label'
#     input_directory = '../Padded-CSVS/'
#     x, y=ReadThreeData(DropColumn,input_directory)
#     x = np.array(x)
#     y = np.array(y)
#     unique,y=encodeLabels(y)
#     no_features = x.shape[1]
#     time_steps = x.shape[2]
#     no_classes = len(unique)
#     print(x.shape)
#     epochs=50
#     batch_size=30
#     ModelSummary(time_steps, no_features, no_classes, 3)
#     Train(x,y,no_features,time_steps,no_classes,epochs,batch_size,3)
# Run()